In [105]:
import pandas as pd

In [106]:
df = pd.read_excel('./Attendance Sheet 2022-2023_Masked.xlsx', sheet_name='Apr 2022')
df.head()

/Users/yulia/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,AtliQ,Unnamed: 1,2022-04-01 00:00:00,2022-04-02 00:00:00,2022-04-03 00:00:00,2022-04-04 00:00:00,2022-04-05 00:00:00,2022-04-06 00:00:00,2022-04-07 00:00:00,2022-04-08 00:00:00,...,Work from home,Paid Leave,Sick Leave,Birthday Leave,Floting festival leave,Bereavement Leave,Leave without pay,Weekly Off,Holidays Off,Menstrual Leave
0,Employee Code,Name,2022-04-01 00:00:00,2022-04-02 00:00:00,2022-04-03 00:00:00,2022-04-04 00:00:00,2022-04-05 00:00:00,2022-04-06 00:00:00,2022-04-07 00:00:00,2022-04-08 00:00:00,...,WFH,PL,SL,BL,FFL,BRL,LWP,WO,HO,ML
1,Atq -406,Thanos Thakur,P,WO,WO,P,P,P,P,P,...,0,0,0,0,0,0,0,9,0,0
2,Atq -462,Jarvis Singh,P,WO,WO,P,P,P,P,P,...,0,0,0,0,0,0,0,9,0,0
3,Atq -411,Nevaeh Waller,P,WO,WO,WFH,P,P,P,P,...,1,0,0,0,0,0,0,9,0,0
4,Atq -398,Bo Cordova,P,WO,WO,P,P,P,WFH,PL,...,3,6,0,0,0,0,0,9,0,1


In [107]:
def process_dataframe(input_df):
    # Rename columns for clarity
    input_df.columns = ['Employee Code', 'Name'] + list(input_df.columns[2:-10]) + ['Work from home', 'Paid Leave', 'Sick Leave', 'Birthday Leave', 
                                                                                   'Floating festival leave', 'Bereavement Leave', 'Leave without pay', 
                                                                                   'Weekly Off', 'Holidays Off', 'Menstrual Leave']

    # Drop specific columns
    columns_to_drop = ['Work from home', 'Paid Leave', 'Sick Leave', 'Birthday Leave', 
                       'Floating festival leave', 'Bereavement Leave', 'Leave without pay', 
                       'Weekly Off', 'Holidays Off', 'Menstrual Leave', "Total Present Days", "Present"]

    input_df.drop(columns=columns_to_drop, inplace=True)

    # Drop the first row if it contains unnecessary header
    if input_df.index[0] == 0:
        input_df = input_df.drop(0).reset_index(drop=True)

    def is_date_column(col):
        try:
            pd.to_datetime(col, format='%Y-%m-%d %H:%M:%S')
            return True
        except (ValueError, TypeError):
            return False

    date_columns = [col for col in input_df.columns if is_date_column(col)]

    # Melt the DataFrame to reshape it
    id_vars = ["Employee Code", "Name"]
    melted_df = pd.melt(input_df, id_vars=id_vars, value_vars=date_columns, var_name='Date', value_name='Value')

    # Convert 'Date' column to datetime type
    melted_df['Date'] = pd.to_datetime(melted_df['Date'], errors='coerce')

    # Drop rows with invalid dates
    melted_df = melted_df.dropna(subset=['Date'])
    melted_df = melted_df.dropna(subset=['Value'])

    return melted_df

In [108]:
processed_df = process_dataframe(df)
processed_df.head()

,Employee Code,Name,Date,Value
0,Atq -406,Thanos Thakur,2022-04-01,P
1,Atq -462,Jarvis Singh,2022-04-01,P
2,Atq -411,Nevaeh Waller,2022-04-01,P
3,Atq -398,Bo Cordova,2022-04-01,P
4,Atq -438,Mekhi Singleton,2022-04-01,P


In [109]:
# Step 1: Read the Excel file
excel_file = './Attendance Sheet 2022-2023_Masked.xlsx'
xls = pd.ExcelFile(excel_file)

# Step 2: Process each sheet and store processed DataFrames
processed_dfs = {}
sheet_names = xls.sheet_names

for sheet_name in sheet_names[:-1]:  # Process all sheets except the last one
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    processed_df = process_dataframe(df)  # Assuming you have defined process_dataframe(df) function
    processed_dfs[sheet_name] = processed_df

# Process the last sheet
last_sheet_name = sheet_names[-1]
last_df = pd.read_excel(excel_file, sheet_name=last_sheet_name)
processed_last_df = last_df  

# Step 3: Save all processed DataFrames into a new Excel file
with pd.ExcelWriter('./Processed_Attendance_Sheets.xlsx', engine='openpyxl') as writer:
    for sheet_name, processed_df in processed_dfs.items():
        processed_df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    processed_last_df.to_excel(writer, sheet_name=last_sheet_name, index=False)

# Inform the user that all sheets have been processed and saved
print("All sheets processed and saved successfully, including the last one.")


/Users/yulia/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/var/folders/_n/w2m9rm9d20vf3mt43wv0tb500000gn/T/ipykernel_15447/816118487.py:29: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  melted_df = pd.melt(input_df, id_vars=id_vars, value_vars=date_columns, var_name='Date', value_name='Value')
/Users/yulia/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/yulia/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


All sheets processed and saved successfully, including the last one.
